In [ ]:
from flask import Flask, request, jsonify

import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout ,Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization
import keras.backend.tensorflow_backend as K

application = Flask(__name__)

@application.route("/")
def hello():
    return "Hello goorm!"

# 처음 시작
@application.route("/sayhello",methods=['POST'])
def sayhello():
    req = request.get_json()    
    sayhello_type = req["action"]["detailParams"]["Sayhello_type"]["value"]	# json파일 읽기
    answer = sayhello_type
    
    # 답변 텍스트 설정
    res = {
        "version": "2.0",
        "template": {
            "outputs": [
                {
                    "simpleText": {
                        "text": answer
                    }
                }
            ]
        }
    }

    # 답변 전송
    return jsonify(res)

# 이미지 분석 모델을 위한 데이터 
def getData(filname):
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

# 이미지 감정 분석 모델
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model

# 사진으로 감정분석
def analysis(img):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
    y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
    y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)
    
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)

    x /= 255
    
    model = my_model()
    h = model.fit(x = X_train,     
                  y = y_train, 
                  batch_size = 64, 
                  epochs = 20, 
                  verbose = 1, 
                  validation_data = (X_test, y_test),
                  shuffle = True,
                  callbacks=[
                      ModelCheckpoint(filepath=path_model),
                  ]
                 )
    custom = model.predict(x)

    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);
    m=0.000000000000000000001
    a=custom[0]
    for i in range(0,len(a)):
        if a[i]>m:
            m=a[i]
            ind=i
    return objects[ind]


# 감정분석 결과
@application.route('/faceAnalysis', methods=['POST'])
def faceAnalysis(img):
    req = request.get_json()    
    sendimage_type = req["action"]["detailParams"]["secureimage"]["value"]	# json파일 읽기
    emotion = analysis(sendimage_type)
    
    # 답변 텍스트 설정
    res = {
        "version": "2.0",
        "template": {
            "outputs": [
                {
                    "simpleText": {
                        "text": "오늘 솔룩이의 기분은 "+emotion+"인 것 같아~"
                    }
                }
            ]
        }
    }

    # 답변 전송
    return jsonify(res)
        
    
if __name__ == "__main__":
    application.run(host='0.0.0.0', port=5000, threaded=True)